# Creating a dictionary from the subset text file.

In [2]:
#Getting most helpful with highest rating + most helpful with lowest ratings.
#Getting percentage of each rating.

import string
import re
from nltk.corpus import stopwords
from stemming.porter2 import stem
import networkx

fhr = open('amazon-meta-subset-new.txt', 'r+', encoding='utf-8', errors='ignore')

# initialize a nested product dictionary that will hold cleaned up amazon product data
# key = ASIN; value = MetaData associated with ASIN
amazonProducts = {}

# read the data from the amazon-meta file and populate amazonProducts nested dicitonary.
(Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, PeopleRating, Rating_Percent) = \
    ("", "", "", "", "", "", 0, 0, 0.0,"", "")

for line in fhr:
    line = line.strip()
    # a product block started
    if(line.startswith("Id")):
        Id = line[3:].strip()
    elif(line.startswith("ASIN")):
        ASIN = line[5:].strip()
    elif(line.startswith("title")):
        Title = line[6:].strip()
        Title = ' '.join(Title.split())
    elif(line.startswith("group")):
        Group = line[6:].strip()
    elif(line.startswith("salesrank")):
        SalesRank = line[10:].strip()
    elif(line.startswith("similar")):
        ls = line.split()
        Copurchased = ' '.join([c for c in ls[2:]])
    elif(line.startswith("categories")):
        ls = line.split()
        Categories = ' '.join((fhr.readline()).lower() for i in range(int(ls[1].strip())))
        Categories = re.compile('[%s]' % re.escape(string.digits+string.punctuation)).sub(' ', Categories)
        Categories = ' '.join(set(Categories.split()))        
        Categories = ' '.join(stem(word) for word in Categories.split())
    elif(line.startswith("reviews")):
        ls = line.split()
        TotalReviews = int(ls[2].strip())
        AvgRating = float(ls[7].strip())
        #percentage of each rating
        each_rating = [0]*5
        Ratings = []
        per_rating = []
        #helpfulness + low + high rating for each helpful 
        tot_reviews_down = int(ls[4].strip())
        helpful_high = 0
        helpful_low = 0
        high_rating = 0
        low_rating = 0
        stringy = ""
        stringy2 = ""
        for i in range(0, tot_reviews_down):
            liney = fhr.readline()
            liney = liney.strip()
            split_line = liney.split(" ")
            split_line = list(filter(None, split_line))
            #for helpfulness + low + high reviews:
            if tot_reviews_down == 0:
                high_rating = 0
                helpful_high = 0
            elif tot_reviews_down == 1:
                high_rating = int(split_line[4])
                helpful_high = int(split_line[8])
            elif tot_reviews_down == 2 or tot_reviews_down>2:
                if int(split_line[4]) >= AvgRating and int(split_line[8]) > helpful_high:
                    helpful_high = int(split_line[8])
                    high_rating = int(split_line[4])
                if int(split_line[4]) < AvgRating and int(split_line[8]) >= helpful_low:
                    helpful_low = int(split_line[8])
                    low_rating = int(split_line[4])
            stringy = "high-rating=" + str(high_rating) + " helpfulness=" + str(helpful_high)
            if low_rating != 0:
                stringy = stringy+ ", low-rating=" + str(low_rating) + " helpfulness=" + str(helpful_low)
                
            #For getting percentage of ratings in the reviews:
            if int(split_line[4]) == 1:
                each_rating[0] += 1
            elif int(split_line[4]) == 2:
                each_rating[1] += 1
            elif int(split_line[4]) == 3:
                each_rating[2] += 1
            elif int(split_line[4]) == 4:
                each_rating[3] += 1
            elif int(split_line[4]) == 5:
                each_rating[4] += 1
                
        for i in range(0, len(each_rating)):
            if tot_reviews_down != 0:
                percent = round((each_rating[i]/tot_reviews_down*100), 2)
                ans = "Rating "+str(i+1) + " occurs " + str(percent) + "%. "
            stringy2 = stringy2 + ans
            
        per_rating.append(stringy2)    
        Ratings.append(stringy)#for helpfulness + low + high ratings 
    # a product block ended
    # write out fields to amazonProducts Dictionary
    elif (line==""):
        try:
            MetaData = {}
            if (ASIN != ""):
                amazonProducts[ASIN]=MetaData
            MetaData['Id'] = Id            
            MetaData['Title'] = Title
            MetaData['Categories'] = ' '.join(set(Categories.split()))
            MetaData['Group'] = Group
            MetaData['Copurchased'] = Copurchased
            MetaData['SalesRank'] = int(SalesRank)
            MetaData['TotalReviews'] = int(TotalReviews)
            MetaData['AvgRating'] = float(AvgRating)
            MetaData['PeopleRating'] = ' '.join(set(Ratings))
            MetaData['Rating_Percent'] = ''.join(set(per_rating))
        except NameError:
            continue
        (Id, ASIN, Title, Categories, Group, Copurchased, SalesRank, TotalReviews, AvgRating, PeopleRating, Rating_Percent) = \
            ("", "", "", "", "", "", 0, 0, 0.0,"", "")
fhr.close()

print(amazonProducts) #Printing the dictionary.

{'0771044445': {'Id': '0', 'Title': '', 'Categories': '', 'Group': '', 'Copurchased': '', 'SalesRank': 0, 'TotalReviews': 0, 'AvgRating': 0.0, 'PeopleRating': 'high-rating=5 helpfulness=21, low-rating=1 helpfulness=7', 'Rating_Percent': 'Rating 1 occurs 16.67%. Rating 2 occurs 0.0%. Rating 3 occurs 0.0%. Rating 4 occurs 41.67%. Rating 5 occurs 41.67%. '}, '0827229534': {'Id': '1', 'Title': 'Patterns of Preaching: A Sermon Sampler', 'Categories': 'christian sermon clergi religion subject preach spiritu book', 'Group': 'Book', 'Copurchased': '0804215715 156101074X 0687023955 0687074231 082721619X', 'SalesRank': 396585, 'TotalReviews': 2, 'AvgRating': 5.0, 'PeopleRating': 'high-rating=5 helpfulness=9', 'Rating_Percent': 'Rating 1 occurs 0.0%. Rating 2 occurs 0.0%. Rating 3 occurs 0.0%. Rating 4 occurs 0.0%. Rating 5 occurs 100.0%. '}, '0738700797': {'Id': '2', 'Title': 'Candlemas: Feast of Flames', 'Categories': 'witchcraft religion subject wicca spiritu book earth base', 'Group': 'Book',

# Question 1: What are the percentages of each rating digit for the product with id: 21?

In [12]:
id = str(21)
for key1 in amazonProducts.keys():
    if amazonProducts[key1]['Id'] == id:
        print("Percentage of each rating: {}".format(amazonProducts[key1]['Rating_Percent']))

Percentage of each rating: Rating 1 occurs 2.86%. Rating 2 occurs 2.14%. Rating 3 occurs 2.14%. Rating 4 occurs 21.43%. Rating 5 occurs 71.43%. 


# Question 2: For products with ASIN: 1573229326  display the most helpful review(s) of the highest rating and the most helpful review(s) of the lowest rating.

In [5]:
# Printing the percentage of each rating and the highest and lowest ratings that are most helpful 
#for a particular ASIN number. 

#Checking for ASIN number: 1573229326
asin = str(1573229326)
for key1 in amazonProducts.keys():
    if key1 == asin:
        print("The highest & lowest rating and their helpfulness: {}".format(amazonProducts[key1]['PeopleRating']))



The highest & lowest rating and their helpfulness: high-rating=4 helpfulness=35, low-rating=2 helpfulness=41


# Question 3: Printing IDs, ASINs, groups and categories that each product belongs to.

In [6]:
#Q7: Group & Category a product belongs to:
for key1 in amazonProducts.keys():
    print("ASIN: {} has ID: {}".format(key1, amazonProducts[key1]["Id"]))
    for key2 in amazonProducts[key1].keys():
        if key2 == "Group":
            print("The product is a: {}".format(amazonProducts[key1]["Group"]))
        if key2 == "Categories":
            print("The categories this object belongs to are: {}".format(amazonProducts[key1]["Categories"]))
    print("\n")

ASIN: 0771044445 has ID: 0
The categories this object belongs to are: 
The product is a: 


ASIN: 0827229534 has ID: 1
The categories this object belongs to are: christian sermon clergi religion subject preach spiritu book
The product is a: Book


ASIN: 0738700797 has ID: 2
The categories this object belongs to are: witchcraft religion subject wicca spiritu book earth base
The product is a: Book


ASIN: 0486287785 has ID: 3
The categories this object belongs to are: general hobbi subject garden book home craft
The product is a: Book


ASIN: 0842328327 has ID: 4
The categories this object belongs to are: religion refer new discipleship subject text translat live sacr studi applic general life guid bibl spiritu other christian histori commentari book testament
The product is a: Book


ASIN: 1577943082 has ID: 5
The categories this object belongs to are: worship busi christian religion devot subject live spiritu book prayerbook
The product is a: Book


ASIN: 0486220125 has ID: 6
The categ